<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>
<h1 align="right">KSO Tutorials #4: Add new clips to a Zooniverse workflow</h1>
<h3 align="right">Written by @jannesgg and @vykanton</h3>
<h5 align="right">Last updated: Nov 25th, 2021</h5>

# Set up and requirements

### Import Python packages

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Set the directory of the libraries
import sys
sys.path.append('..')

# Set to display dataframes as interactive tables
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)

# Import required modules
import utils.tutorials_utils as t_utils
import utils.t4_utils as t4
#import utils.server_utils as server_utils

print("Packages loaded successfully")

<IPython.core.display.Javascript object>

Packages loaded successfully


### Choose your project

In [3]:
project = t_utils.choose_project()

Dropdown(description='Project:', options=('Koster_Seafloor_Obs', 'Spyfish_Aotearoa', 'SGU'), value='Koster_Sea…

### Initiate sql and zoo project

In [4]:
# Initiate db
db_info_dict = t_utils.initiate_db(project.value)

Enter your username for SNIC server········
Enter your password for SNIC server········
Updated sites
Updated movies
Updated species


In [7]:
# Connect to Zooniverse project
zoo_project = t_utils.connect_zoo_project(project.value)

Enter your Zooniverse user········
Enter your Zooniverse password········


# Retrieve info about zooniverse clips

In [8]:
zoo_info_dict = t_utils.retrieve__populate_zoo_info(project_name = project.value, 
                                                    db_info_dict = db_info_dict,
                                                    zoo_project = zoo_project,
                                                    zoo_info = ["subjects"])

Retrieving subjects from Zooniverse
subjects were retrieved successfully
Updated subjects
The database has a total of 2342 frame subjects and 7362 clip subjects have been updated


# Retrieve info about movies hosted in the server

In [18]:
# Store info about the movies available in the server
available_movies_df = t4.retrieve_movie_info_from_server(project_name = project.value,
                                                         db_info_dict = db_info_dict)

95 movies are mapped from the server


### Select the movie you want to upload to Zooniverse

In [19]:
movie_i = t4.movie_to_upload(available_movies_df = available_movies_df)

Combobox(value='', description='Movie to upload:', ensure_option=True, layout=Layout(width='50%'), options=('0…

### Check if movie is already in Zooniverse

Remember to query the newest zooniverse data to get the most up to date list of clips uploaded

In [ ]:
# Check movie hasn't been uploaded to Zooniverse
t4.check_movie_uploaded(movie_i = movie_i.value,
                        db_info_dict = db_info_dict)

### Specify the number of clips and clip length

In [ ]:
clip_selection = t4.select_clip_n_len(movie_i = movie_i.value,
                                     db_info_dict = db_info_dict)

### Review the clips that will be created

In [ ]:
t4.review_clip_selection(clip_selection = clip_selection, 
                         movie_i = movie_i.value)

### Create the clips

In [ ]:
clips_to_upload_df = t4.create_clips(available_movies_df = available_movies_df, 
                                     movie_i = movie_i.value,
                                     db_info_dict = db_info_dict,
                                     clip_selection = clip_selection,  
                                     project_name = project.value)

Make sure your workflows in Zooniverse have different names to avoid issues while selecting the workflow id

### Review clips

In [ ]:
t4.check_clip_size(clip_paths = clips_to_upload_df["clip_path"].unique())

### Video modifications

In [ ]:
# Work in progress
clip_modification = t4.select_modification()

In [ ]:
# Write the clip modification details. See ffmpeg documentation for instructions (http://ffmpeg.org/ffmpeg.html)
modification_details = {
  "-crf": "30",
  "-c:v": "libx264",
  #"-vf": "curves=red=0/0 0.396/0.67 1/1:green=0/0 0.525/0.451 1/1:blue=0/0 0.459/0.517 1/1,scale=1280:-1",#borrowed from https://www.element84.com/blog/color-correction-in-space-and-at-sea                         
  "-pix_fmt": "yuv420p",
  "-preset": "veryfast"
}

In [ ]:
# Modify the clips
clips_ready_df = t4.modify_clips(clips_to_upload_df = clips_to_upload_df[175:179].reset_index(drop=True), 
                                 movie_i = movie_i.value,
                                 clip_modification = clip_modification.value, 
                                 modification_details = modification_details)

In [ ]:
# Review the size of the modified clips
t4.check_clip_size(clip_paths = clips_ready_df["modif_clip_path"].unique())

In [ ]:
# Compare the original and modified clips
t4.compare_clips(df = clips_ready_df)

### Set Zooniverse metadata

In [ ]:
upload_to_zoo, sitename, created_on = t4.set_zoo_metadata(df = clips_ready_df,
                                                          project_name = project.value)

In [ ]:
#upload_to_zoo = upload_to_zoo.fillna("unknown")

### Upload clips to Zooniverse

You may receive an error message related to file size if clips exceed the recommended limit for Zooniverse uploads. In this case, we recommend shortening the clip length to achieve a suitable filesize.

In [ ]:
t4.upload_clips_to_zooniverse(upload_to_zoo = upload_to_zoo, 
                              sitename = sitename,
                              created_on = created_on,
                              project = zoo_clips_info_dict["project"])

In [ ]:
#END